# Step 3: Train Llama 3 on Your Data
Run this notebook in Google Colab to fine-tune Llama 3 using the dataset you generated.

**Steps:**
1. Go to `Runtime` -> `Change runtime type` -> select `T4 GPU`.
2. Upload your `yourbook_finetune.jsonl` file to the Colab file browser.
3. Run all cells below.

In [ ]:
%%capture
# 1. Install Unsloth (Makes training 2x faster and fit in free Colab GPU)
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [ ]:
# 2. Load the Model (Llama 3 8B)
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 
dtype = None 
load_in_4bit = True # Use 4bit quantization to reduce memory usage

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Provide this function to format your data
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

In [ ]:
# 3. Load Your Dataset
from datasets import load_dataset

# REPLACE THIS with the name of the file you uploaded
dataset_filename = "Itihas va Rajyashastra Book_finetune.jsonl" 

dataset = load_dataset("json", data_files = dataset_filename, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
# 4. Setup Training
from trl import SFTTrainer
from transformers import TrainingArguments

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Increase this for better results (e.g. 100-300)
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
# 5. Train!
trainer_stats = trainer.train()

In [ ]:
# 6. Save as GGUF (for Ollama)
# This will convert the model and save it as 'model.gguf'
model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

In [ ]:
# 7. Download the File
from google.colab import files
files.download("model/unsloth.Q4_K_M.gguf")